## DIA 031: Implementacion de WebSockets para Actualizaciones en Tiempo Real

Integración de WebSockets:

Utilizaste la extensión Flask-SocketIO para habilitar comunicación bidireccional en tiempo real entre el servidor y los clientes.
Configuraste el servidor para emitir mensajes de estado y actualizaciones instantáneas, lo que mejora la experiencia del usuario al permitir una interacción dinámica.
Actualización del Frontend:

Modificaste el frontend para que se conecte al servidor de WebSocket.
Implementaste lógica para mostrar notificaciones y actualizaciones en tiempo real, como el progreso de predicciones o cambios en el estado de la aplicación.
Mejora de la Observabilidad:

La integración de WebSockets facilita la supervisión en tiempo real de eventos importantes y puede usarse para enviar alertas instantáneas a los administradores o usuarios finales.
Permite una respuesta más rápida a incidencias, ya que las actualizaciones se reciben en el momento en que ocurren.
Consideraciones de Escalabilidad y Seguridad:

Se revisaron aspectos de seguridad para asegurar que la conexión WebSocket esté protegida, utilizando HTTPS y mecanismos de autenticación.
Se consideraron estrategias para escalar el servicio de WebSockets en entornos de alta concurrencia.
Beneficios Obtenidos:

Experiencia de Usuario Mejorada: Los usuarios reciben información en tiempo real sin tener que recargar la página.
Interacción Dinámica: Permite una comunicación más fluida y bidireccional entre el cliente y el servidor.
Respuesta Inmediata: Facilita la detección y respuesta ante eventos críticos, mejorando la capacidad de monitoreo y la gestión de incidencias.
Recomendaciones para Continuar:

Seguridad de la Conexión: Asegurarse de que la conexión WebSocket esté cifrada y protegida, implementando HTTPS y autenticación robusta.
Optimización y Escalabilidad: Investigar sobre estrategias de escalado para el servicio de WebSockets (como clustering o uso de servicios gestionados) si la cantidad de conexiones concurrentes aumenta significativamente.
Monitoreo en Tiempo Real: Integrar las actualizaciones de WebSocket con las herramientas de monitoreo existentes para tener una visión consolidada del rendimiento y estado de la aplicación.


CODIGO DEL SERVIDOR (api.py):
from flask import Flask, render_template, request, jsonify, url_for
from flask_socketio import SocketIO, emit
import time
import threading
import logging

# Configuración básica de Flask
app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'  # Cambia esto a una clave segura

# Inicialización de Flask-SocketIO
socketio = SocketIO(app, cors_allowed_origins="*")

# Configurar logging (opcional, para ver mensajes en la consola)
logging.basicConfig(level=logging.INFO)

# Ruta principal que carga la interfaz
@app.route('/')
def index():
    return render_template('index.html')

# Endpoint para iniciar una tarea larga (simulada)
@app.route('/start_long_task', methods=['POST'])
def start_long_task():
    # Inicia una tarea en segundo plano que simula un proceso largo
    thread = threading.Thread(target=long_task)
    thread.start()
    return jsonify({"msg": "Tarea iniciada"}), 200

# Función que simula una tarea larga y envía actualizaciones de progreso
def long_task():
    for progress in range(1, 101):
        # Emitir una actualización de progreso a todos los clientes conectados
        socketio.emit('task_update', {'progress': progress})
        time.sleep(0.1)  # Simula tiempo de procesamiento
    # Una vez completada la tarea, emitir un mensaje final
    socketio.emit('task_complete', {'msg': '¡La tarea se completó exitosamente!'})

# Eventos de conexión y desconexión de WebSocket
@socketio.on('connect')
def handle_connect():
    app.logger.info('Cliente conectado')
    emit('connection', {'data': 'Conectado al servidor'})

@socketio.on('disconnect')
def handle_disconnect():
    app.logger.info('Cliente desconectado')

# Ejecutar la aplicación
if __name__ == '__main__':
    socketio.run(app, debug=True)


## CODIGO DEL CLIENTE (templates/index.html)

<!DOCTYPE html>
<html>
<head>
    <title>WebSocket en Tiempo Real</title>
    <!-- Cargar la librería de Socket.IO -->
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.5.4/socket.io.js"></script>
    <!-- jQuery para facilitar las operaciones AJAX -->
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
</head>
<body>
    <h1>Actualizaciones en Tiempo Real</h1>
    
    <!-- Botón para iniciar la tarea larga -->
    <button id="startTask">Iniciar Tarea Larga</button>
    
    <!-- Área para mostrar el progreso -->
    <div id="progress">Progreso: 0%</div>
    
    <!-- Área para mostrar el mensaje final -->
    <div id="complete"></div>
    
    <script>
        // Conectar al servidor de WebSocket
        var socket = io();

        // Al conectarse, mostrar un mensaje en consola
        socket.on('connect', function() {
            console.log('Conectado al servidor WebSocket');
        });

        // Recibir mensaje de conexión
        socket.on('connection', function(msg) {
            console.log(msg.data);
        });

        // Escuchar actualizaciones de la tarea
        socket.on('task_update', function(data) {
            $('#progress').text('Progreso: ' + data.progress + '%');
        });

        // Escuchar la finalización de la tarea
        socket.on('task_complete', function(data) {
            $('#complete').text(data.msg);
        });

        // Función para iniciar la tarea larga mediante una solicitud POST
        $('#startTask').click(function() {
            $.post('/start_long_task', function(response) {
                console.log(response.msg);
            });
        });
    </script>
</body>
</html>
